# <center> PROCESAMIENTO DIGITAL DE SEÑALES DE AUDIO</center>
## <center>Mel-frequency cepstral coefficients (MFCCs)</center>      

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

import librosa
import librosa.display

import IPython.display as ipd

librosa.__version__

**NOTA:** *Las siguientes dos celdas solo son necesarias para descargar el archivo de ejemplo. Ignórelas si va a trabajar con sus propios archivos de audio.*

In [ ]:
!pip install wget

In [ ]:
import wget

### Cómo correr el notebook
Se puede bajar y correr el notebook de forma local en una computadora.

O también se puede correr en Google Colab usando el siguiente enlace. 

<table align="center">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/mrocamora/audio-dsp/blob/main/notebooks/audioDSP-MFCC_example.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

### Parte 1 - cálculo de los MFCCs
A continuación se siguen los siguientes pasos para el cálculo de los Coeficientes Cepstrales de Frecuencia Mel (MFCCs).

1. Cálculo del espectrograma (de potencia).
2. Aplicación de banco de filtros en escala Mel.
3. Cálculo del logaritmo (potencia a dB)
4. Cálculo de la DCT y se devuelven primeros coeficientes (liftrado)

También se calcula y muestra el banco de filtros en escala Mel. 

In [ ]:
# download audio file
wget.download('https://github.com/mrocamora/audio-dsp/blob/main/audio/superstition.wav?raw=true')

In [ ]:
# read the audio file
filename = 'superstition.wav'

y, sr = librosa.load(filename)

In [ ]:
# plot audio signal
plt.figure(figsize=(12,8))
ax1 = plt.subplot(2, 1, 1)
librosa.display.waveplot(y, sr=sr)
plt.title('audio waveform')
plt.tight_layout()

In [ ]:
ipd.Audio(y, rate=sr)

In [ ]:
# 1. Compute spectrogam from STFT
n_fft = 2048
Y = librosa.stft(y, win_length=1024, hop_length=512, n_fft=n_fft, window='hann')
S = np.abs(Y)**2

# 2. apply mel-filterbank to combine FFT bins into Mel-frequency bins
# number of mel-frequency bands
n_mels = 128
# maximum frequency for the analysis
fmax = 4000 
# compute mel-spectrogram
M = librosa.feature.melspectrogram(S=S, n_mels=n_mels, fmax=fmax)

# 3. apply log to convert power to dB
M_log = librosa.power_to_db(M)

# 4. apply DCT and return first n_mfcc coefficients
# number of MFCC coefficients 
n_mfcc = 20
# compute MFCCs from mel-spectrogram
mfccs = librosa.feature.mfcc(S=M_log, n_mfcc=n_mfcc)

# NOTE: the following function is a wrapper for all of the above
# mfccs = librosa.feature.mfcc(y=y, n_mfcc=n_mfcc)

In [ ]:
# compute and plot the Mel filter bank
melfb = librosa.filters.mel(sr, n_fft, fmax=fmax, n_mels=n_mels)
freqs = librosa.fft_frequencies(n_fft=n_fft)

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
librosa.display.specshow(melfb, x_axis='linear')
plt.xlim([0, fmax])
plt.ylabel('Mel filter')
plt.title('Mel filter bank')
plt.subplot(1, 2, 2)
plt.plot(freqs, melfb.T)
plt.title('Mel filter bank')
plt.xlabel('Frequency [Hz]')
plt.xlim([0, fmax])
plt.tight_layout()

In [ ]:
# plot mel-spectrogram and MFCCs
ind_max = np.argmax(freqs > fmax)
plt.figure(figsize=(12, 8))
plt.subplot(2, 1, 1)
librosa.display.specshow(librosa.power_to_db(S[:ind_max, :]), y_coords=freqs[:ind_max],
                         y_axis='linear')#, x_axis='time')
plt.title('spectrogram')
plt.subplot(2, 1, 2)
librosa.display.specshow(M_log, x_axis='time', y_axis='mel', sr=sr, fmax=fmax)
plt.title('mel-spectrogram')
plt.tight_layout()

In [ ]:
# plot MFCCs
ind = [4, 14]

plt.figure(figsize=(12, 8))
plt.subplot(2, 1, 1)
librosa.display.specshow(mfccs, x_axis='time')
plt.title('MFCC (cofficients ' + str(0) + ' to ' + str(n_mfcc) + ')')
plt.tight_layout()

plt.subplot(2, 1, 2)
librosa.display.specshow(mfccs[ind[0]:ind[1], :], x_axis='time')
plt.title('MFCC (cofficients ' + str(ind[0]) + ' to ' + str(ind[1]) + ')')
plt.tight_layout()

### Parte 2 - reconstrucción de la señal de audio a partir de los MFCCs

Por último, se invierte el proceso de forma de reconstruír una señal de audio a partir de los MFCCs. 

Para ello, se siguen los siguientes pasos. 

1. Primero se obtiene el espectrograma en escala Mel aplicando la transformada DCT inversa e invirtiendo el logaritmo. 
2. Luego se pasa de frecuencias en escala Mel a frecuencia en escala lineal, para obtener un espectrograma tradicional. 
3. Por último, se obtiene una señal de audio a partir del espectrograma, usando una versión rápida del algoritmo Griffin-Lim [1][2].

La señal obtenida permite confirmar que los MFCCs codifican una estimación de la envolvente espectral de la señal de audio.

[1] D. W. Griffin and J. S. Lim, “Signal estimation from modified short-time Fourier transform,” IEEE Trans. ASSP, vol.32, no.2, pp.236–243, Apr. 1984.

[2] Perraudin, N., Balazs, P., & Søndergaard, P. L. “A fast Griffin-Lim algorithm,” IEEE Workshop on Applications of Signal Processing to Audio and Acoustics (pp. 1-4), Oct. 2013.

In [ ]:
# 1. Invert Mel-frequency cepstral coefficients to approximate a Mel power spectrogram.
# Inverse DCT is applied to the MFCCs followed by dB to power spectrum mapping. 
W = librosa.feature.inverse.mfcc_to_mel(mfccs, n_mels=n_mels)

# 2. Approximate STFT magnitude from a Mel power spectrogram.
X = librosa.feature.inverse.mel_to_stft(W)

# 3. Approximate magnitude spectrogram inversion using the “fast” Griffin-Lim algorithm. 
x = librosa.griffinlim(X)

In [ ]:
# plot audio signal
plt.figure(figsize=(12,8))
ax1 = plt.subplot(2, 1, 1)
librosa.display.waveplot(x, sr=sr)
plt.title('audio waveform')
ax2 = plt.subplot(2, 1, 2)
librosa.display.specshow(librosa.power_to_db(X[:ind_max, :]**2), y_coords=freqs[:ind_max],
                         y_axis='linear', x_axis='time')
plt.title('spectrogram')
plt.tight_layout()

In [ ]:
ipd.Audio(x, rate=sr)